## ⚙️ 1️⃣ LLM’de Compile Kavramı

`compile` fonksiyonu, özellikle **Keras/TensorFlow** dünyasında modelin eğitime hazır hale gelmesi için kullanılır. Burada 3 ana şey tanımlanır:

- **Optimizer** – modelin ağırlıklarını güncelleme yöntemi  
  Örnekler: `Adam`, `AdamW`, `SGD`, `Adafactor`  
  LLM’lerde genellikle `AdamW` veya `Adafactor` tercih edilir.

- **Loss Function (Kayıp Fonksiyonu)** – modelin tahmin hatasını ölçer  
  Seq2Seq / LLM için genellikle `CrossEntropyLoss` kullanılır (logit + softmax + target).

- **Metrics (Opsiyonel)** – eğitim sırasında performans izlemek için  
  Örnek: `accuracy`, `perplexity`

> ⚠️ LLM’lerde klasik `accuracy` çoğu zaman yanıltıcıdır, onun yerine **perplexity** veya **token-level accuracy** tercih edilir.

---

## ⚙️ 2️⃣ LLM’de Loss Fonksiyonu

LLM’lerde **loss** genellikle **token-level cross entropy** ile hesaplanır:

\[
\text{Loss} = - \frac{1}{N} \sum_{i=1}^{N} \sum_{j=1}^{V} y_{ij} \log(\hat{y}_{ij})
\]

- \(N\) = batch içindeki token sayısı  
- \(V\) = vocabulary boyutu  
- \(y_{ij}\) = gerçek token one-hot  
- \(\hat{y}_{ij}\) = modelin tahmini token olasılığı  

**Masking:** Padding tokenlerini loss hesaplamaya dahil etmeyiz, aksi halde model yanlış yönlendirilir.


## ⚙️ 3️⃣ PyTorch Örneği

In [ ]:
import torch
import torch.nn as nn
from transformers import AdamW

# Model ve optimizer
model = MySeq2SeqModel(vocab_size=30522, d_model=512)
optimizer = AdamW(model.parameters(), lr=5e-5)

# Loss
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# Eğitim adımı
def train_step(input_ids, target_ids):
    optimizer.zero_grad()
    outputs = model(input_ids, labels=target_ids)
    loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)),
                     target_ids.view(-1))
    loss.backward()
    optimizer.step()
    return loss.item()


### Notlar:

* outputs.logits shape’i [batch, seq_len, vocab_size]

* Flatten işlemi ile [batch*seq_len, vocab_size] yapıyoruz, target da [batch*seq_len]

* ignore_index padding tokenlerini göz ardı eder

## ⚙️ 4️⃣ Gelişmiş Loss Seçenekleri

* Label Smoothing – modelin aşırı confident olmasını engeller

* Focal Loss – nadir tokenler için ağırlık verir

* Perplexity – loss’u anlamlı bir ölçüye çevirir:

## ⚙️ 5️⃣ Özet

| Adım            | Açıklama                                       |
| --------------- | ---------------------------------------------- |
| Optimizer       | Ağırlıkları güncellemek için (AdamW/Adafactor) |
| Loss            | Token-level CrossEntropy, padding maskeli      |
| Metrics         | Token accuracy / perplexity                    |
| Label smoothing | Overfitting önler                              |
| Masking         | Padding tokenleri hariç tutar                  |


----

# 1️⃣ Optimizer’lar (LLM’lerde yaygın)

| Optimizer          | Açıklama                                                                                                                         | Öne çıkan parametreler                                                                              |
| ------------------ | -------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------- |
| **AdamW**          | Adam’ın weight decay (L2 regularization) eklenmiş versiyonu. Transformer tabanlı LLM’lerde en yaygın kullanılan optimizer.       | `lr` (learning rate), `betas` (momentum), `eps` (küçük sabit), `weight_decay`                       |
| **Adafactor**      | RAM verimliliği yüksek, büyük model ve düşük batch size için optimize edilmiş Adam türevi. HuggingFace’de LLM’ler için standart. | `lr`, `eps1`, `eps2`, `clip_threshold`, `relative_step` (learning rate schedule), `scale_parameter` |
| **Adam**           | Klasik Adam, küçük modeller veya prototipler için. LLM’lerde genellikle weight decay için AdamW tercih edilir.                   | `lr`, `betas`, `eps`                                                                                |
| **SGD / Momentum** | Nadir kullanılır, genellikle LLM’lerde tercih edilmez.                                                                           | `lr`, `momentum`, `weight_decay`                                                                    |


# 2️⃣ Loss Fonksiyonları (LLM / Seq2Seq)

| Loss Fonksiyonu                    | Açıklama                                                                                                 | Öne çıkan parametreler                                                                |
| ---------------------------------- | -------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------- |
| **CrossEntropyLoss (token-level)** | Token başına softmax + log + negatif log-likelihood. Seq2Seq ve LLM’lerde en standart loss.              | `ignore_index` (padding tokenleri ignore etmek için), `reduction` (`mean` veya `sum`) |
| **LabelSmoothingCrossEntropy**     | CrossEntropy’nin label smoothing ile geliştirilmiş versiyonu. Modelin aşırı confident olmasını engeller. | `smoothing` (örn. 0.1), `ignore_index`, `reduction`                                   |
| **Focal Loss**                     | Nadir tokenler veya dengesiz token dağılımları için ağırlık verir.                                       | `gamma`, `alpha`, `ignore_index`                                                      |
| **KLDivLoss / Perplexity**         | Genellikle dağılım farklılıklarını ölçmek veya distillation için kullanılır.                             | `log_target`, `reduction`                                                             |


# 3️⃣ Parametrelerin Tipik Ayarları (Büyük LLM Örneği)

| Parametre      | Tipik Değer                                            |
| -------------- | ------------------------------------------------------ |
| `lr`           | 1e-4 … 5e-5 (warmup ile)                               |
| `weight_decay` | 0.01 … 0.1                                             |
| `betas`        | (0.9, 0.999)                                           |
| `eps`          | 1e-8                                                   |
| `smoothing`    | 0.1 (label smoothing için)                             |
| `ignore_index` | pad_token_id (genellikle 0 veya tokenizer özel değeri) |


### 💡 Özet Mantık:

* Optimizer: Model ağırlıklarını stabil ve verimli güncellemek için → AdamW veya Adafactor.

* Loss: Her tokenin hatasını ölçmek için → token-level CrossEntropy, isteğe bağlı label smoothing.

* Parametreler: Learning rate, weight decay, betas, eps ve padding mask en kritik ayarlar.

# = >

```python 
import torch
import torch.nn as nn
from transformers import AdamW, Adafactor, get_linear_schedule_with_warmup

# -----------------------
# Model Tanımı (Örnek)
# -----------------------
model = MySeq2SeqModel(vocab_size=30522, d_model=512)

# -----------------------
# 1️⃣ Optimizer Seçimi
# -----------------------
# AdamW: LLM'lerde standart
optimizer = AdamW(
    model.parameters(),
    lr=5e-5,          # learning rate
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0.01
)

# Alternatif: Adafactor (hafıza dostu)
# optimizer = Adafactor(
#     model.parameters(),
#     lr=None,
#     relative_step=True,
#     scale_parameter=True,
#     warmup_init=True
# )

# -----------------------
# 2️⃣ Loss Fonksiyonu
# -----------------------
# Padding tokenlerini ignore eden standart CrossEntropyLoss
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# -----------------------
# 3️⃣ Scheduler (Opsiyonel)
# -----------------------
num_training_steps = 10000
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=500,
    num_training_steps=num_training_steps
)

# -----------------------
# 4️⃣ Train Step (Compile-like)
# -----------------------
def train_step(input_ids, target_ids):
    model.train()
    optimizer.zero_grad()

    # Model forward
    outputs = model(input_ids, labels=target_ids)

    # logits: [batch, seq_len, vocab_size], target: [batch, seq_len]
    loss = criterion(
        outputs.logits.view(-1, outputs.logits.size(-1)),
        target_ids.view(-1)
    )

    loss.backward()
    optimizer.step()
    scheduler.step()  # opsiyonel
    return loss.item()

# -----------------------
# 5️⃣ Metric Örnek
# -----------------------
def perplexity(loss):
    return torch.exp(loss)
